In [3]:
import sklearn
import numpy as np
from sklearn.datasets import load_breast_cancer

In [5]:
data = load_breast_cancer()
X = data.data
y = data.target
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
m = len(X)
print(f"Training Examples: {m}")

X shape: (569, 30)
y shape: (569,)
Training Examples: 569


In [7]:
def H(p):
    if p==0 or p==1:
        return 0
    H_p = -p*np.log2(p)-((1-p)*np.log2(1-p))
    return H_p

In [8]:
def compute_entropy(y):
    p1 = np.mean(y==1)
    return H(p1)

In [9]:
print(f"Entropy of Root Node: {compute_entropy(y)}")

Entropy of Root Node: 0.9526351224018599


In [24]:
def split_dataset(X,node_indices,feature,threshold):
    left_indices = []
    right_indices = []
    for i in node_indices:
        if(X[i][feature] <= threshold):
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices,right_indices

In [26]:
root_indices = [0,1,2,3,4,5,6,7,8,9,540,541,542,543,544,545]
feature = 0
threshold = np.median(X[root_indices, feature])
left_indices, right_indices = split_dataset(X, root_indices, feature, threshold)
print(f"Left Indice: {left_indices}")
print(f"Right Indice: {right_indices}")

Left Indice: [3, 5, 7, 8, 9, 540, 543, 545]
Right Indice: [0, 1, 2, 4, 6, 541, 542, 544]


In [29]:
def compute_information_gain(X,y,node_indices,feature,threshold):
    left_indices, right_indices = split_dataset(X,node_indices,feature,threshold)
    y_node = y[node_indices]
    y_left = y[left_indices]
    y_right = y[right_indices]
    information_gain = 0
    root_entropy = compute_entropy(y_node)
    left_entropy = compute_entropy(y_left)
    right_entropy = compute_entropy(y_right)
    w_left = len(y_left) / len(y_node)
    w_right = len(y_right) / len(y_node)
    weighted_entropy = w_left*left_entropy + w_right*right_entropy
    information_gain = root_entropy - weighted_entropy
    return information_gain

In [30]:
root_indices = list(range(20))  # just first 20 samples
feature = 0
threshold = np.median(X[root_indices, feature])

ig = compute_information_gain(X, y, root_indices, feature, threshold)
print("Information Gain:", ig)

Information Gain: 0.05189916032131564


In [35]:
def best_split(X, y, node_indices):
    best_feature = None
    best_threshold = None
    best_ig = -1
    
    n_features = X.shape[1]
    for feature in range(n_features):
        values = X[node_indices, feature]
        unique_values = np.unique(values)

        if len(values) == 1:
            continue
        thresholds = (unique_values[:-1] + unique_values[1:]) / 2.0

        for threshold in thresholds:
            ig = compute_information_gain(X, y, node_indices, feature, threshold)
            
            if ig > best_ig:
                best_ig = ig
                best_feature = feature
                best_threshold = threshold
    
    return best_feature, best_threshold, best_ig


In [36]:
root_indices = list(range(30))  # small subset of samples
feature, threshold, ig = best_split(X, y, root_indices)

print("Best feature:", feature)
print("Best threshold:", threshold)
print("Information gain:", ig)

Best feature: 6
Best threshold: 0.070295
Information gain: 0.36082517699473016


In [38]:
def build_tree_recursive(X, y, node_indices, branch, max_depth, current_depth=0):
    y_node = y[node_indices]
    
    # Stopping condition 1: max depth reached
    if current_depth == max_depth:
        print("  " * current_depth + f"- {branch} leaf node with indices {node_indices}")
        return {"type": "leaf", "class": int(np.bincount(y_node).argmax())}
    
    # Stopping condition 2: pure node (all labels same)
    if np.all(y_node == y_node[0]):
        print("  " * current_depth + f"- {branch} pure leaf with class {y_node[0]}")
        return {"type": "leaf", "class": int(y_node[0])}
    
    # Find best split
    best_feature, best_threshold, best_ig = best_split(X, y, node_indices)
    
    # Stopping condition 3: no information gain
    if best_feature is None or best_ig <= 0:
        print("  " * current_depth + f"- {branch} leaf (no gain) with indices {node_indices}")
        return {"type": "leaf", "class": int(np.bincount(y_node).argmax())}
    
    print("  " * current_depth + f"- Depth {current_depth}, {branch}: Split on feature {best_feature} at threshold {best_threshold:.3f}")
    
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, best_feature, best_threshold)
    
    # Build subtrees
    left_subtree = build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    right_subtree = build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)
    
    # Return decision node
    return {
        "type": "node",
        "feature": best_feature,
        "threshold": best_threshold,
        "left": left_subtree,
        "right": right_subtree
    }


In [39]:
root_indices = list(range(20))
tree = build_tree_recursive(X, y, root_indices, "Root", max_depth=3)

- Depth 0, Root: Split on feature 6 at threshold 0.070
  - Depth 1, Left: Split on feature 0 at threshold 14.780
    - Left pure leaf with class 1
    - Right pure leaf with class 0
  - Right pure leaf with class 0


In [40]:
def predict_sample(tree, x):
    if tree["type"] == "leaf":
        return tree["class"]
    
    # Decision node
    feature = tree["feature"]
    threshold = tree["threshold"]
    
    if x[feature] <= threshold:
        return predict_sample(tree["left"], x)
    else:
        return predict_sample(tree["right"], x)

In [41]:
def predict(tree, X):
    return np.array([predict_sample(tree, x) for x in X])

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build tree (example with max depth 3)
root_indices = list(range(len(X_train)))
tree = build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=3)

# Predict on test set
y_pred = predict(tree, X_test)

# Evaluate accuracy
print("Test Accuracy:", accuracy_score(y_test, y_pred))

- Depth 0, Root: Split on feature 7 at threshold 0.051
  - Depth 1, Left: Split on feature 20 at threshold 16.830
    - Depth 2, Left: Split on feature 10 at threshold 0.626
      - Left leaf node with indices [0, 2, 3, 4, 7, 8, 10, 13, 14, 18, 19, 23, 26, 28, 29, 31, 35, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 56, 57, 58, 60, 61, 63, 64, 66, 67, 68, 72, 73, 75, 77, 78, 79, 80, 82, 83, 85, 86, 87, 89, 92, 93, 94, 96, 97, 98, 99, 101, 102, 103, 104, 105, 107, 110, 111, 113, 114, 115, 116, 117, 118, 119, 123, 124, 126, 127, 129, 131, 133, 135, 136, 138, 139, 140, 142, 144, 146, 148, 151, 152, 153, 154, 156, 157, 158, 160, 161, 162, 164, 166, 167, 168, 169, 170, 171, 172, 174, 175, 176, 178, 181, 182, 184, 187, 190, 194, 195, 198, 199, 201, 203, 204, 205, 209, 210, 213, 214, 216, 219, 222, 227, 228, 229, 230, 232, 237, 238, 239, 241, 242, 243, 244, 248, 250, 251, 252, 253, 254, 256, 257, 260, 262, 264, 265, 266, 267, 269, 271, 274, 275, 276, 278, 279, 282, 284, 288, 29

In [44]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X,y)
y_pred = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 1.0
